In [2]:
import geopandas as gpd
from pathlib import Path
import json
import pandas as pd
from sqlalchemy import URL, create_engine, text as sql_text

In [16]:
credentials = Path('../../inputs/db_credentials.json')
with open(credentials, 'r') as f:
    credentials = json.load(f)

user     = credentials['user']
password = credentials['password']
host     = credentials['host']
port     = credentials['port']
database = credentials['database']

In [17]:
url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

# create a SQLAlchemy engine object
engine = create_engine(url_object)

Execute a query

In [5]:
thuc_id = 1002

In [8]:
with engine.connect() as connection:
    try:
        query = f""" 
        ALTER TABLE thuc_{thuc_id}_annagnps_cell_data_section
        ADD COLUMN soil_id_annagnps_valid INT;

        UPDATE thuc_{thuc_id}_annagnps_cell_data_section
        SET soil_id_annagnps_valid = CASE WHEN soil_id IN (SELECT Soil_ID FROM usa_valid_soil_data) THEN 1 ELSE 0 END;

        """
        connection.execute(sql_text(query))
        # Commit the transaction explicitly
        connection.commit()

    except Exception as e:
        print(e)
        # Rollback the transaction in case of an error
        connection.rollback()


Check a query

In [18]:
thuc_id = 1004

In [19]:
query = f"SELECT * FROM thuc_{thuc_id}_annagnps_cell_data_section"

with engine.connect() as conn:

    try:

        df = pd.read_sql_query(sql=sql_text(query), con=conn)

    except Exception as e:

        raise e

In [20]:
df[df['soil_id_annagnps_valid']==1]

,cell_id,soil_id,mgmt_field_id,reach_id,reach_location_code,cell_area,time_of_conc,avg_elevation,rcn_calib_id,avg_land_slope,...,delivery_ratio,constant_usle_c_fctr,constant_usle_p_fctr,all_oc_calib_fctr,all_n_calib_fctr,all_p_calib_fctr,sheet_and_rill_erosion_calib_fctr,gullies_erosion_calib_fctr,input_units_code,soil_id_annagnps_valid
0,186002,1428407,None,18600,1,12.96,None,222.13,None,0.00805,...,None,None,None,None,None,None,None,None,1,1
1,186001,1428407,None,18600,0,10.08,None,222.80,None,0.00607,...,None,None,None,None,None,None,None,None,1,1
2,186003,1428407,None,18600,1,0.90,None,221.27,None,0.00872,...,None,None,None,None,None,None,None,None,1,1
3,185992,1428407,None,18599,1,23.40,None,220.48,None,0.00610,...,None,None,None,None,None,None,None,None,1,1
4,185983,1428407,None,18598,1,23.58,None,220.51,None,0.00637,...,None,None,None,None,None,None,None,None,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69830,51,211638,None,5,0,15.03,None,126.38,None,0.04076,...,None,None,None,None,None,None,None,None,1,1
69831,31,211642,None,3,0,15.03,None,127.61,None,0.06534,...,None,None,None,None,None,None,None,None,1,1
69832,32,211685,None,3,1,15.39,None,119.16,None,0.04934,...,None,None,None,None,None,None,None,None,1,1
69833,23,1543947,None,2,1,37.80,None,105.44,None,0.03020,...,None,None,None,None,None,None,None,None,1,1
